# ANÁLISE DIMENSIONAL

In [1]:
#======================================================#
# 1. IMPORTING NECESSARY CLASSES AND DIMENSIONAL MATRIX#
#======================================================#

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.linalg as sc
import scipy.stats as st
from MRPy import *

# Importing pandas dataframe with dimension exponents for relevant quantities

DimData = pd.read_excel('resources/DimData.xlsx', 
                         index_col  =  0,
                         header     =  0,
                         sheet_name = 'DimData')


In [2]:
DimData

,descriptor,latex,L,M,T
a,Acceleration,a,1,0,-2
α,Angular acceleration,\alpha,0,0,-2
ω,Angular frequency,\omega,0,0,-1
A,Area,A,2,0,0
EI,Beam bending stiffness,EI,3,1,-2
GAs,Beam shear stiffness,GA_s,1,1,-2
ρ,Density,\rho,-3,1,0
μ,Dynamic viscosity,\mu,-1,1,-1
F,Force,F,1,1,-2
q,Force per unit length,q,0,1,-2


In [3]:
#======================================================#
# 2. MAKING DIMENSIONAL MATRIX                         #
#======================================================#
ABC    = ['L', 'ρ', 'f']           
LMT    = ['L', 'M', 'T']           
base   =  DimData.loc[ABC, LMT]    
i_base =  np.linalg.inv(base)      

print(base)

   L  M  T
L  1  0  0
ρ -3  1  0
f  0  0 -1


In [80]:
#======================================================#
# 3. DEFINING NEW SCALES                               #
#======================================================#


λ_L    =  1/406.4         # length scale for the reduced model
λ_ρ    =  1/1          # acceleration remains unchanged (same gravity)
λ_f   =  5.67/0.2       # imposed flexural stiffness scale


In [81]:
#======================================================#
# 4. MAKING NEW DIMENSIONAL BASE MATRIX                #
#======================================================#

par    = ['a','v', 'EI','m','L']         # selected scales to be calculated
npar   =  len(par)                           # number of quantities
DimMat =  DimData.loc[par, LMT]              # the dimensional matrix

print(DimMat)

    L  M  T
a   1  0 -2
v   1  0 -1
EI  3  1 -2
m   0  1  0
L   1  0  0


In [82]:
scales =  np.tile([λ_L, λ_ρ, λ_f],(npar,1))  # prepare for calculations

NewMat =  pd.DataFrame(data    = np.matmul(DimMat.values, i_base),
                       index   = DimMat.index,
                       columns = ABC)
print(NewMat)

      L    ρ    f
a   1.0  0.0  2.0
v   1.0  0.0  1.0
EI  6.0  1.0  2.0
m   3.0  1.0  0.0
L   1.0  0.0  0.0


In [83]:
#======================================================#
# 5. PRINTING NECESSARY SCALES                         #
#======================================================#
[λ_a,λ_v, λ_EI, λ_m,λ_L]  = np.prod(scales**NewMat, axis=1);

print('Aceleração:       λ_a   =  1 : {0:5.15f}'.format(λ_a),  '\n'
      'Velocidade:       λ_v   =  1 : {0:5.15f}'.format(λ_v),  '\n'     
      'Rigidez:          λ_EI  =  1 : {0:2.15f}'.format(λ_EI), '\n'
      'Massa:            λ_m   =  1 : {0:2.15f}'.format(λ_m), '\n'
      'Comprimento:      λ_L   =  1 : {0:2.15f}'.format(λ_L)) 

Aceleração:       λ_a   =  1 : 1.977663631889763 
Velocidade:       λ_v   =  1 : 0.069758858267717 
Rigidez:          λ_EI  =  1 : 0.000000000000178 
Massa:            λ_m   =  1 : 0.000000014898375 
Comprimento:      λ_L   =  1 : 0.002460629921260


In [84]:
print('Aceleração:             1 / λ_a  =  {0:4.5f}'.format(1/λ_a),  '\n'
      'Velocidade:             1 / λ_v  =  {0:4.2f}'.format(1/λ_v), '\n'
      'Rigidez à flexão:       1 / λ_EI =  {0:4.0f}'.format(1/λ_EI), '\n'
      'Massa:                  1 / λ_m  =  {0:4.1f}'.format(1/λ_m), '\n'
      'Comprimento:            1 / λ_L  =  {0:4.1f}'.format(1/λ_L))

Aceleração:             1 / λ_a  =  0.50565 
Velocidade:             1 / λ_v  =  14.34 
Rigidez à flexão:       1 / λ_EI =  5605522100837 
Massa:                  1 / λ_m  =  67121414.1 
Comprimento:            1 / λ_L  =  406.4


In [92]:
ρ = 160       # [ kg/m³ ]

hp = 182.88   # [ m ] 
l1p = 30.48   # [ m ] 
l2p = 45.72   # [ m ] 

volume = hp*l1p*l2p

m = ρ*volume

print('Massa do Protótipo [CAARC] :          {0:5.4f} kg'.format(m))

Massa do Protótipo [CAARC] :          40776259.0925 kg


In [123]:
print('Altura do CAARC:          {0:5.2f} cm'.format(182.88*λ_L*100))
print('Largura do CAARC:         {0:5.2f} cm'.format(30.48*λ_L*100))
print('Comprimento do CAARC:     {0:5.2f} cm'.format(45.72*λ_L*100))


print('Massa Total do Modelo:     {0:5.2f} g'.format(m*λ_m*1000))

Altura do CAARC:          45.00 cm
Largura do CAARC:          7.50 cm
Comprimento do CAARC:     11.25 cm
Massa Total do Modelo:     607.50 g


In [124]:
massa_total_escalada = m*λ_m*1000
m_placa = np.array((31., 31. ,31., 31., 31.))
m_arame = np.array((13.,13.,13.,13.))
m_chumbinho = np.array((10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,
                        10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.,10.))
mi_placa = np.mean(m_placa)
s_placa = np.std(m_placa)

mi_arame = np.mean(m_arame)
s_arame = np.std(m_arame)

mi_chumbinho = np.mean(m_chumbinho)
s_chumbinho = np.std(m_chumbinho)

print('Massa média da placa de acrílico (laje):                     {0:5.2f} g'.format(mi_placa))
print('Desvio Padrão da massa da placa de acrílico (laje):          {0:5.2f} g'.format(s_placa))
print()
print('Massa média do fio de arame (coluna):                     {0:5.2f} g'.format(mi_arame))
print('Desvio Padrão da massa do fio de arame (coluna):          {0:5.2f} g'.format(s_arame))
print()
print('Massa média do chumbinho:                     {0:5.2f} g'.format(mi_chumbinho))
print('Desvio Padrão da massa do chumbinho:          {0:5.2f} g'.format(s_chumbinho))

Massa média da placa de acrílico (laje):                     31.00 g
Desvio Padrão da massa da placa de acrílico (laje):           0.00 g

Massa média do fio de arame (coluna):                     13.00 g
Desvio Padrão da massa do fio de arame (coluna):           0.00 g

Massa média do chumbinho:                     10.00 g
Desvio Padrão da massa do chumbinho:           0.00 g


In [132]:
massa_total_calculada = 4*mi_arame + 5*mi_placa + 40*mi_chumbinho
massa_restante = massa_total_escalada - massa_total_calculada

print('Massa Total calculada do Modelo Reduzido :           {0:5.2f} g'.format(massa_total_calculada))
print('Erro absoluto da massa :                             {0:5.2f} g'.format(massa_restante))
print('Erro relativo da massa :                             {0:5.5f} %'.format(100*massa_restante/massa_total_escalada))

Massa Total calculada do Modelo Reduzido :           607.00 g
Erro absoluto da massa :                              0.50 g
Erro relativo da massa :                             0.08230 %


In [136]:

print('Velocidade básica do Vento (Vo) no Túnel de Vento:          {0:5.3f} m/s'.format(45*λ_v))

Velocidade básica do Vento (Vo) no Túnel de Vento:          3.139 m/s
